### Dogs data web scrapping

In [9]:
import pandas as pd
import numpy as np
import time
from selenium import webdriver

### Get  XLS 

In [10]:
df = pd.read_excel('dogs_breeds_rename.xlsx')
df.head()

,breed,breed_web
0,Chihuahua,chihuahua
1,Japanese_spaniel,japanese-chin
2,Maltese_dog,maltese
3,Pekinese,pekingese
4,Shih-Tzu,shih-tzu


### Functions for web Scrapping

In [11]:
def get_origin(browser):
    h3s = browser.find_elements_by_tag_name('h3')
    for h3 in h3s:
        text = h3.text
        if text == 'AREA OF ORIGIN':
            # Get parent DIV
            div = h3.find_element_by_xpath('..')
            p_origin = div.find_element_by_tag_name('p')
            return p_origin.text

In [12]:
def get_height(browser):
    h3s = browser.find_elements_by_tag_name('h3')
    for h3 in h3s:
        text = h3.text
        if text == 'HEIGHT':
            # Get parent DIV
            div = h3.find_element_by_xpath('..')
            p_height = div.find_element_by_tag_name('p')
            return p_height.text

In [13]:
def get_weight(browser):
    h3s = browser.find_elements_by_tag_name('h3')
    for h3 in h3s:
        text = h3.text
        if text == 'WEIGHT':
            # Get parent DIV
            div = h3.find_element_by_xpath('..')
            p_weight = div.find_element_by_tag_name('p')
            return p_weight.text

In [14]:
def get_life_span(browser):
    lis = browser.find_elements_by_tag_name('li')
    for li in lis:
        text = li.text
        if 'Life span' in text:
            texts = text.split(':')
            return texts[1]

In [15]:
def get_temperament(browser):
    h2 = browser.find_element_by_id('temperamentSection')
    div = h2.find_element_by_xpath('..')
    p = div.find_element_by_tag_name('p')
    return p.text

In [16]:
def get_health(browser):
    list_health = ''
    h2 = browser.find_element_by_id('healthSection')
    div = h2.find_element_by_xpath('..')

    try:
        ul = div.find_element_by_tag_name('ul')
        lis = ul.find_elements_by_tag_name('li')

        for li in lis:
            list_health += '|' + li.text

        return list_health
    except:
        p = div.find_element_by_tag_name('p')
        print(p.text)
        return p.text

In [17]:
list_dogs_parameter = df['breed_web'].values.tolist()
url_base = 'https://www.petfinder.com/dog-breeds/'
browser = webdriver.Firefox()
columns = ['dog_race','origin','height','weight','life_span','temperament','health']
frames = list()

for dog in list_dogs_parameter:
    load_page = False
    while(load_page == False):
        try:
            time.sleep(1)
            if type(dog) == str:
                full_url = url_base + dog + '/'
                browser.get(full_url)
                origin = get_origin(browser)
                height = get_height(browser)
                weight = get_weight(browser)
                life_span = get_life_span(browser)
                temperament = get_temperament(browser)
                health = get_health(browser)
            else:
                origin = 'NA'
                height = 'NA'
                weight = 'NA'
                life_span = 'NA'
                temperament = 'NA'
                health = 'NA'
            new_row = {'dog_race':[dog],'origin':[origin],
                       'height':[height],'weight':[weight],
                       'life_span':[life_span],'temperament':[temperament],
                       'health':[health]}

            df_row = pd.DataFrame(new_row)
            frames.append(df_row)
           
            load_page = True
        except:
            load_page = False


browser.quit()
result = pd.concat(frames)

• Major concerns: none • Minor concerns: lens luxation, distichiasis • Occasionally seen: Legg–Perthes, vWD • Suggested tests: eye, (vWD) • Life span: 12–16 years
Major Concerns: N/A


In [18]:
result.head()

,dog_race,origin,height,weight,life_span,temperament,health
0,chihuahua,Mexico,"6-9""",6 lb,14–18 years,The saucy Chihuahua has earned his place as a ...,|Major concerns: none|Minor concerns: pulmonic...
0,japanese-chin,Japan,"8-11""",4-7 lb,10–12 years,"The Japanese Chin is a devoted companion, reli...","|Minor concerns: patellar luxation, KCS, entro..."
0,maltese,Malta,"9-11""",4-6 lb,12–14 years,"Long a favorite lapdog, the gentle Maltese fil...",|Major concerns: none|Minor concerns: patellar...
0,pekingese,China,"6-9""",14 lb,13–15 years,The Pekingese is decidedly not a timid lapdog....,|Major concerns: none|Minor concerns: elongate...
0,shih-tzu,"China, Tibet","8-11""",9-16 lb,11–14 years,The spunky but sweet Shih Tzu can be both a ge...,|Major concerns: CHD|Minor concerns: renal dys...


In [19]:
result.to_csv('dogs_data.csv')

In [21]:
dogs_data = pd.read_csv('dogs_data.csv')

In [22]:
dogs_data.head()

,Unnamed: 0,dog_race,origin,height,weight,life_span,temperament,health
0,0,chihuahua,Mexico,"6-9""",6 lb,14–18 years,The saucy Chihuahua has earned his place as a ...,|Major concerns: none|Minor concerns: pulmonic...
1,0,japanese-chin,Japan,"8-11""",4-7 lb,10–12 years,"The Japanese Chin is a devoted companion, reli...","|Minor concerns: patellar luxation, KCS, entro..."
2,0,maltese,Malta,"9-11""",4-6 lb,12–14 years,"Long a favorite lapdog, the gentle Maltese fil...",|Major concerns: none|Minor concerns: patellar...
3,0,pekingese,China,"6-9""",14 lb,13–15 years,The Pekingese is decidedly not a timid lapdog....,|Major concerns: none|Minor concerns: elongate...
4,0,shih-tzu,"China, Tibet","8-11""",9-16 lb,11–14 years,The spunky but sweet Shih Tzu can be both a ge...,|Major concerns: CHD|Minor concerns: renal dys...


### Cleaning final csv

In [30]:
df['breed'] = [x.replace("_"," ").title() for x in df['breed'].values.tolist()]

In [34]:
dogs_data['breed_name_id'] = df['breed'].values.tolist()

In [36]:
dogs_data.tail()

,Unnamed: 0,dog_race,origin,height,weight,life_span,temperament,health,breed_name_id
115,0,poodle,"Germany, Central Europe","21""",45-65 lb,10–13 years,Among the very smartest and most obedient of d...,"|Major concerns: sebaceous adenitis, gastric t...",Standard Poodle
116,0,xoloitzcuintli-mexican-hairless,Mexico,"10-23""",9-35 lb,11–14 years,"As a primitive breed, Xoloitzcuintli may be so...",|Major concerns: none|Minor concerns: acne|Occ...,Mexican Hairless
117,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dingo
118,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dhole
119,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,African Hunting Dog


In [37]:
dogs_data.columns

Index(['Unnamed: 0', 'dog_race', 'origin', 'height', 'weight', 'life_span',
       'temperament', 'health', 'breed_name_id'],
      dtype='object')

### Remove unnecessary columns

In [38]:
del dogs_data['Unnamed: 0']

In [40]:
del dogs_data['dog_race']

In [43]:
dogs_data.head()

,origin,height,weight,life_span,temperament,health,breed_name_id
0,Mexico,"6-9""",6 lb,14–18 years,The saucy Chihuahua has earned his place as a ...,|Major concerns: none|Minor concerns: pulmonic...,Chihuahua
1,Japan,"8-11""",4-7 lb,10–12 years,"The Japanese Chin is a devoted companion, reli...","|Minor concerns: patellar luxation, KCS, entro...",Japanese Spaniel
2,Malta,"9-11""",4-6 lb,12–14 years,"Long a favorite lapdog, the gentle Maltese fil...",|Major concerns: none|Minor concerns: patellar...,Maltese Dog
3,China,"6-9""",14 lb,13–15 years,The Pekingese is decidedly not a timid lapdog....,|Major concerns: none|Minor concerns: elongate...,Pekinese
4,"China, Tibet","8-11""",9-16 lb,11–14 years,The spunky but sweet Shih Tzu can be both a ge...,|Major concerns: CHD|Minor concerns: renal dys...,Shih-Tzu


### Add Ids

In [48]:
ids = [x + 1 for x in range(120)]

In [50]:
dogs_data['id'] = ids

In [51]:
dogs_data.head()

,origin,height,weight,life_span,temperament,health,breed_name_id,id
0,Mexico,"6-9""",6 lb,14–18 years,The saucy Chihuahua has earned his place as a ...,|Major concerns: none|Minor concerns: pulmonic...,Chihuahua,1
1,Japan,"8-11""",4-7 lb,10–12 years,"The Japanese Chin is a devoted companion, reli...","|Minor concerns: patellar luxation, KCS, entro...",Japanese Spaniel,2
2,Malta,"9-11""",4-6 lb,12–14 years,"Long a favorite lapdog, the gentle Maltese fil...",|Major concerns: none|Minor concerns: patellar...,Maltese Dog,3
3,China,"6-9""",14 lb,13–15 years,The Pekingese is decidedly not a timid lapdog....,|Major concerns: none|Minor concerns: elongate...,Pekinese,4
4,"China, Tibet","8-11""",9-16 lb,11–14 years,The spunky but sweet Shih Tzu can be both a ge...,|Major concerns: CHD|Minor concerns: renal dys...,Shih-Tzu,5


### Save Clean CSV

In [52]:
dogs_data.to_csv('dogs_clean_data.csv')